In [1]:
import os
import pathlib
import numpy as np
from PIL import Image
import tensorflow as tf

In [2]:
# os.system('unzip -oq CatsDogs.zip')

In [3]:
# Define the directory of the dataset
data_dir = pathlib.Path('CatsDogs/')

# Remove corrupted files
os.system("rm CatsDogs/Cats/666.jpg CatsDogs/Dogs/11702.jpg CatsDogs/Dogs/11410.jpg")

# Collects the path of all the files within the dataset
data_paths = [str(path) for path in list(data_dir.glob("*/*.jpg"))]
print(f"Images in the dataset: {len(data_paths)}")

Images in the dataset: 24997


rm: CatsDogs/Cats/666.jpg: No such file or directory
rm: CatsDogs/Dogs/11702.jpg: No such file or directory
rm: CatsDogs/Dogs/11410.jpg: No such file or directory


In [4]:
# Convert non-jpeg images into jpeg files
# formats = [(path, Image.open(path).format) for path in data_paths]
# non_jpegs = list(filter(lambda x: x[1]!='JPEG', formats))
# for path, _ in non_jpegs:
#    img = Image.open(path)
#    img.convert('RGB').save(path, format='JPEG')

In [5]:
# Create the respective tf.data.Dataset object
dataset = tf.data.Dataset.from_tensor_slices(data_paths)
# Shuffle the dataset
dataset = dataset.shuffle(len(data_paths), reshuffle_each_iteration=False)

Metal device set to: Apple M1 Pro


2022-07-19 18:55:02.410202: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-19 18:55:02.410332: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
# Get the class names
class_names = np.array(sorted([item.name for item in data_dir.glob('*') if item.name[0] != '.']))
print(class_names)

['Cats' 'Dogs']


In [7]:
# Spilt the dataset
test_size = int(len(list(dataset)) * 0.2)
train = dataset.skip(test_size)
test = dataset.take(test_size)

# Create a validation set
val_size = int(len(list(train))*0.2)
val = train.take(val_size)
train = train.skip(val_size)

In [8]:
# Set initial params for the loader
batch_size = 64
img_height = 150
img_width = 150

In [9]:
def get_label(file_path):
    # Convert the path to a list of path components
    parts = tf.strings.split(file_path, os.path.sep)
    # The second to last is the class-directory
    one_hot = parts[-2] == class_names
    # Integer encode the label
    return tf.argmax(one_hot)

In [10]:
def decode_img(img):
    # Convert the compressed string to a 3D uint8 tensor
    img = tf.io.decode_jpeg(img, channels=3)
    # Resize the image to the desired size
    return tf.image.resize(img, [img_height, img_width])

In [11]:
def process_path(file_path):
    label = get_label(file_path)
    # Load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

In [12]:
# Create a dataset of image, label pairs
train = train.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
test = test.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
val = val.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)

In [13]:
# Configure dataset for performance
def configure_for_performance(ds):
    ds = ds.cache()
    ds = ds.shuffle(buffer_size=1000)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)
    return ds

In [14]:
train = configure_for_performance(train)
test = configure_for_performance(test)
val = configure_for_performance(val)

In [15]:
model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.25),
    # tf.keras.layers.Dense(128, activation='leaky_relu'),
    # tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [16]:
 model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(16, 3, activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


In [17]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [18]:
n_epochs = 20
model.fit(
    train,
    epochs=n_epochs,
    validation_data=val
)

Epoch 1/20


2022-07-19 18:55:03.755200: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-07-19 18:55:03.755345: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
Corrupt JPEG data: 252 extraneous bytes before marker 0xd9


 98/250 [==========>...................] - ETA: 6s - loss: 0.6810 - accuracy: 0.5915

Corrupt JPEG data: 128 extraneous bytes before marker 0xd9
Corrupt JPEG data: 162 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1153 extraneous bytes before marker 0xd9


120/250 [=============>................] - ETA: 5s - loss: 0.6718 - accuracy: 0.6027

...

221/250 [=========================>....] - ETA: 1s - loss: 0.6328 - accuracy: 0.6417

Corrupt JPEG data: 1403 extraneous bytes before marker 0xd9


250/250 [==============================] - ETA: 0s - loss: 0.6232 - accuracy: 0.6514

2022-07-19 18:55:15.154122: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
Corrupt JPEG data: 99 extraneous bytes before marker 0xd9
Corrupt JPEG data: 65 extraneous bytes before marker 0xd9
Corrupt JPEG data: 2226 extraneous bytes before marker 0xd9
Corrupt JPEG data: 214 extraneous bytes before marker 0xd9


250/250 [==============================] - 13s 49ms/step - loss: 0.6232 - accuracy: 0.6514 - val_loss: 0.5358 - val_accuracy: 0.7252
Epoch 2/20
250/250 [==============================] - 11s 44ms/step - loss: 0.5120 - accuracy: 0.7449 - val_loss: 0.4865 - val_accuracy: 0.7614
Epoch 3/20
250/250 [==============================] - 11s 45ms/step - loss: 0.4367 - accuracy: 0.7946 - val_loss: 0.4360 - val_accuracy: 0.7899
Epoch 4/20
250/250 [==============================] - 11s 44ms/step - loss: 0.3776 - accuracy: 0.8308 - val_loss: 0.4512 - val_accuracy: 0.7904
Epoch 5/20
250/250 [==============================] - 11s 44ms/step - loss: 0.3226 - accuracy: 0.8569 - val_loss: 0.4304 - val_accuracy: 0.8095
Epoch 6/20
250/250 [==============================] - 11s 44ms/step - loss: 0.2655 - accuracy: 0.8880 - val_loss: 0.4512 - val_accuracy: 0.8077
Epoch 7/20
250/250 [==============================] - 11s 44ms/step - loss: 0.1998 - accuracy: 0.9196 - val_loss: 0.4912 - val_accuracy: 0.8077
Epo

In [19]:
def zero_one_loss(dataset, dataset_size):
    
    _, accuracy = model.evaluate(dataset)
    zero_one_loss = dataset_size*(1-accuracy)

    return int(round(zero_one_loss, ndigits=0))

In [20]:
print(f"Zero-one loss on the test set: {zero_one_loss(test, test_size)}")

79/79 [==============================] - 2s 20ms/step - loss: 1.1761 - accuracy: 0.8146
Zero-one loss on the test set: 927


In [21]:
# Clear any previous state
# del train, test, model, dataset, test_size, formats, non_jpegs
del model
tf.keras.backend.clear_session()

In [22]:
import gc
gc.collect()

120

# K-fold cross validation

In [23]:
"""
from sklearn.model_selection import KFold


k_fold = KFold(n_splits=5, shuffle=True)
k_splits = k_fold.split(data_paths)
results = []

for train_index, test_index in k_splits:

    # Get the paths to the data
    train_paths = np.asarray(data_paths)[train_index]
    test_paths = np.asarray(data_paths)[test_index]

    # Make it tf.data.Dataset
    train = tf.data.Dataset.from_tensor_slices(train_paths)
    test = tf.data.Dataset.from_tensor_slices(test_paths)

    # Shuffle the dataset
    train = train.shuffle(len(train))
    test = test.shuffle(len(test))

    # Get labels
    train = train.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
    test = test.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
    
    # Configure for performance
    train = configure_for_performance(train)
    test = configure_for_performance(test)

    # Create the model
    model = tf.keras.Sequential([
        tf.keras.layers.Rescaling(1./255),
        tf.keras.layers.Conv2D(16, 3, activation='relu', input_shape=(150, 150, 3)),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Conv2D(32, 3, activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Conv2D(64, 3, activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    model.fit(
        train,
        epochs=n_epochs,
        verbose=0,
    )

    loss = zero_one_loss(test, len(test_paths))
    results.append(loss)
    print(f"Zero-one loss: {loss}")

    # Clear any previous state
    del model
    tf.keras.backend.clear_session()
    gc.collect()
"""

'\nfrom sklearn.model_selection import KFold\n\n\nk_fold = KFold(n_splits=5, shuffle=True)\nk_splits = k_fold.split(data_paths)\nresults = []\n\nfor train_index, test_index in k_splits:\n\n    # Get the paths to the data\n    train_paths = np.asarray(data_paths)[train_index]\n    test_paths = np.asarray(data_paths)[test_index]\n\n    # Make it tf.data.Dataset\n    train = tf.data.Dataset.from_tensor_slices(train_paths)\n    test = tf.data.Dataset.from_tensor_slices(test_paths)\n\n    # Shuffle the dataset\n    train = train.shuffle(len(train))\n    test = test.shuffle(len(test))\n\n    # Get labels\n    train = train.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)\n    test = test.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)\n    \n    # Configure for performance\n    train = configure_for_performance(train)\n    test = configure_for_performance(test)\n\n    # Create the model\n    model = tf.keras.Sequential([\n        tf.keras.layers.Rescaling(1./255),\n        tf.ke

In [24]:
""" 
mean_loss = np.round(np.mean(results), decimals=0)
std_loss = np.round(np.std(results), decimals=0)
print(f'The mean of zero-one loss is {int(mean_loss)}, with a standard deviation of {int(std_loss)} missmatched samples')
"""

" \nmean_loss = np.round(np.mean(results), decimals=0)\nstd_loss = np.round(np.std(results), decimals=0)\nprint(f'The mean of zero-one loss is {int(mean_loss)}, with a standard deviation of {int(std_loss)} missmatched samples')\n"